In [15]:
import csv
import re
from re import search
import pandas as pd
import sys
#!{sys.executable} -m pip install us
!{sys.executable} -m pip install git+git://github.com/jmbielec/geograpy3.git 
from geograpy3 import extraction
import us

  Cloning git://github.com/jmbielec/geograpy3.git to /private/var/folders/3j/_qhynnn53ksghddxthvnwm8h0000gn/T/pip-req-build-v78rmxad
  Running command git clone -q git://github.com/jmbielec/geograpy3.git /private/var/folders/3j/_qhynnn53ksghddxthvnwm8h0000gn/T/pip-req-build-v78rmxad
  fatal: remote error:
    The unauthenticated git protocol on port 9418 is no longer supported.
  Please see https://github.blog/2021-09-01-improving-git-protocol-security-github/ for more information.
ERROR: Command errored out with exit status 128: git clone -q git://github.com/jmbielec/geograpy3.git /private/var/folders/3j/_qhynnn53ksghddxthvnwm8h0000gn/T/pip-req-build-v78rmxad Check the logs for full command output.


In [16]:
df = pd.read_csv('data/step2_preprocessed_bios.csv')
df


,index,user_id,user_location,user_bio,hashtag,tokens,Segmented#
0,0,3.436262e+08,USA,#TrueToAtlanta...ATL Braves 21 🏆 ATLUTD⭐️ ...2...,"['TrueToAtlanta', 'GoDawgs']",atl brave atlutd orange bowl playoffs,true to atlanta go dawgs
1,1,1.180000e+18,"Cleveland, OH","I'm a nerdy guy who loves art, music, wrestlin...",[],nerdy guy love art music wrestle comics poly,NaN
2,2,1.440000e+18,Harrisburg Pa,No worries Lil Ms Sunshine cause I don't mess ...,[],worry lil sunshine cause mess woman moon civil...,NaN
3,3,3.169218e+09,"Houston, TX, USA, The Ghetto","Fuck yo feelings!\nFollow me please, I follow ...","['lgbtq', 'feminist', 'blacklivesmatter', 'Army']",fuck feel follow follow atl bear nigerian rais...,lgbtq feminist blacklivesmatter army
4,4,1.219720e+08,"Denver, CO",I am a huge sports and music fan. I graduated ...,[],huge sport music fan graduate regis university...,NaN
...,...,...,...,...,...,...,...
2333,4105,2.961228e+08,"Cincinnati, OH",Christian. Husband. Father. Ph.D. Candidate @B...,['ChelseaFC'],christian husband father candidate mosquito re...,chelsea fc
2334,4106,8.800000e+17,"Los Angeles, CA","MarTech, MBA, Foodie, Bilingual, Soccer, Boxin...",['FairPlay'],martech mba foodie bilingual soccer box gamer ...,fair play
2335,4111,1.153059e+09,"Princeton, KY","streams occasionally, will choose race cars ov...",[],stream occasionally choose race cars important...,NaN
2336,4116,2.432869e+08,"Tucson, AZ",Enjoy the practical wisdom of daily life. Prom...,[],enjoy practical wisdom daily life promote trut...,NaN


In [17]:
df["user_location"] = df["user_location"].str.upper()    # capitalizing data in User Location column
df

,index,user_id,user_location,user_bio,hashtag,tokens,Segmented#
0,0,3.436262e+08,USA,#TrueToAtlanta...ATL Braves 21 🏆 ATLUTD⭐️ ...2...,"['TrueToAtlanta', 'GoDawgs']",atl brave atlutd orange bowl playoffs,true to atlanta go dawgs
1,1,1.180000e+18,"CLEVELAND, OH","I'm a nerdy guy who loves art, music, wrestlin...",[],nerdy guy love art music wrestle comics poly,NaN
2,2,1.440000e+18,HARRISBURG PA,No worries Lil Ms Sunshine cause I don't mess ...,[],worry lil sunshine cause mess woman moon civil...,NaN
3,3,3.169218e+09,"HOUSTON, TX, USA, THE GHETTO","Fuck yo feelings!\nFollow me please, I follow ...","['lgbtq', 'feminist', 'blacklivesmatter', 'Army']",fuck feel follow follow atl bear nigerian rais...,lgbtq feminist blacklivesmatter army
4,4,1.219720e+08,"DENVER, CO",I am a huge sports and music fan. I graduated ...,[],huge sport music fan graduate regis university...,NaN
...,...,...,...,...,...,...,...
2333,4105,2.961228e+08,"CINCINNATI, OH",Christian. Husband. Father. Ph.D. Candidate @B...,['ChelseaFC'],christian husband father candidate mosquito re...,chelsea fc
2334,4106,8.800000e+17,"LOS ANGELES, CA","MarTech, MBA, Foodie, Bilingual, Soccer, Boxin...",['FairPlay'],martech mba foodie bilingual soccer box gamer ...,fair play
2335,4111,1.153059e+09,"PRINCETON, KY","streams occasionally, will choose race cars ov...",[],stream occasionally choose race cars important...,NaN
2336,4116,2.432869e+08,"TUCSON, AZ",Enjoy the practical wisdom of daily life. Prom...,[],enjoy practical wisdom daily life promote trut...,NaN


In [18]:
df["user_location"] = df["user_location"].fillna("UNKNOWN")
df

,index,user_id,user_location,user_bio,hashtag,tokens,Segmented#
0,0,3.436262e+08,USA,#TrueToAtlanta...ATL Braves 21 🏆 ATLUTD⭐️ ...2...,"['TrueToAtlanta', 'GoDawgs']",atl brave atlutd orange bowl playoffs,true to atlanta go dawgs
1,1,1.180000e+18,"CLEVELAND, OH","I'm a nerdy guy who loves art, music, wrestlin...",[],nerdy guy love art music wrestle comics poly,NaN
2,2,1.440000e+18,HARRISBURG PA,No worries Lil Ms Sunshine cause I don't mess ...,[],worry lil sunshine cause mess woman moon civil...,NaN
3,3,3.169218e+09,"HOUSTON, TX, USA, THE GHETTO","Fuck yo feelings!\nFollow me please, I follow ...","['lgbtq', 'feminist', 'blacklivesmatter', 'Army']",fuck feel follow follow atl bear nigerian rais...,lgbtq feminist blacklivesmatter army
4,4,1.219720e+08,"DENVER, CO",I am a huge sports and music fan. I graduated ...,[],huge sport music fan graduate regis university...,NaN
...,...,...,...,...,...,...,...
2333,4105,2.961228e+08,"CINCINNATI, OH",Christian. Husband. Father. Ph.D. Candidate @B...,['ChelseaFC'],christian husband father candidate mosquito re...,chelsea fc
2334,4106,8.800000e+17,"LOS ANGELES, CA","MarTech, MBA, Foodie, Bilingual, Soccer, Boxin...",['FairPlay'],martech mba foodie bilingual soccer box gamer ...,fair play
2335,4111,1.153059e+09,"PRINCETON, KY","streams occasionally, will choose race cars ov...",[],stream occasionally choose race cars important...,NaN
2336,4116,2.432869e+08,"TUCSON, AZ",Enjoy the practical wisdom of daily life. Prom...,[],enjoy practical wisdom daily life promote trut...,NaN


In [20]:
#In this cell I am trying to use the states dictionary to change the "User Location" to state abbreviation.
import geograpy3

def replace_states(bio):
    region = []
    count = 0
    # replace each state with its abbreviation
    for cell in bio:
        x = geograpy3.get_place_context(text = cell)
        region_cell = x.places
        region.append(region_cell)
        print(region_cell)
        count += 1
        print(count)
# return the modified string (or original if no states were found)
    return region


            
df['user_location2'] = replace_states(df['user_location'])   
df['user_location2'] 

[]
1
['CLEVELAND', 'OH']
2
['HARRISBURG']
3
['HOUSTON', 'TX', 'USA', 'THE', 'GHETTO']
4
['DENVER', 'CO']
5
[]
6
['UTAH']
7
['NEW YORK', 'UNITED STATES']
8
[]
9
['OHIO']
10
['ORLEANS', 'LA']
11
['SAN', 'FERNANDO', 'CA']
12
['SAN', 'FRANCISCO', 'CA']
13
['OCEANSIDE', 'CA']
14
['MOUNDVILLE', 'AL']
15
['GOODYEAR', 'AZ']
16
['INTERNATIONAL', 'SPACE']
17
['RENO', 'NEVADA']
18
['NAPA', 'CA']
19
['HUNTINGTON', 'BEACH', 'CA']
20
['SAN', 'DIEGO', 'CA']
21
[]
22
['SKAGIT', 'COUNTY', 'WASHINGTON']
23
[]
24
['NEW YORK']
25
[]
26
['COLUMBUS', 'OH']
27
['THIBODAUX', 'LA']
28
['SALT', 'LAKE', 'UTAH']
29
[]
30
['NC']
31
[]
32
['CORVALLIS', 'OR']
33
['OTTAWA', 'CANADA']
34
['UNITED STATES']
35
['LOS ANGELES', 'CA']
36
['SAN', 'ANTONIO', 'TX']
37
['GLEN', 'BURNIE', 'MD']
38
['PROBABLY', 'WITH']
39
['OAKLAND', 'CA']
40
['DALLAS', 'TX']
41
['SAN', 'FRANCISCO', 'JAMMIN']
42
['CALIFORNIA', 'USA']
43
['FL']
44
['WASHINGTON', 'USA']
45
['NEWPORT', 'OR']
46
['BROOKLYN', 'NY']
47
['BELLEVILLE', 'ONTARIO']
48
['L

['WASHINGTON', 'DC']
401
[]
402
['WASHINGTON']
403
['TEXAS']
404
['HTWN']
405
['SAN', 'DIEGO']
406
[]
407
['YVR']
408
['VIETNAM', 'YOW', 'CANADA']
409
['LOS ANGELES', 'CA']
410
['LOUISVILLE', 'KENTUCKY']
411
['CEDAR', 'RAPIDS', 'IA']
412
['CORPUS', 'CHRISTI', 'TEXAS']
413
['SAN', 'ANTONIO', 'TX']
414
['BRONX', 'NY']
415
['TUCSON', 'AZ']
416
['PICKENS', 'SC']
417
['AKRON', 'OH']
418
['MO']
419
['TEMA', 'GHANA']
420
['GEORGIA', 'USA']
421
['SACRAMENTO', 'CA']
422
['BELLFLOWER', 'CA']
423
['LOUISVILLE', 'KY']
424
[]
425
['SAN', 'DIEGO', 'CA']
426
['LOS ANGELES', 'CA']
427
[]
428
['HOLLYWOOD', 'LOS']
429
[]
430
['COLLEGE', 'STATION', 'TX']
431
['LOS ANGELES', 'CA']
432
['AUSTIN', 'TEXAS']
433
['NEWPORT', 'BEACH', 'CA', 'LONGSOCAL']
434
['EARTH']
435
['SEATTLE', 'WA']
436
['SEATTLE']
437
['BRITISH']
438
['SAN', 'DIEGO', 'CA']
439
['SAN', 'FRANCISCO', 'CA']
440
['PALM', 'BEACH', 'FL']
441
['PHOENIX', 'AZ']
442
['TORONTO']
443
['ATLANTA', 'GA']
444
['CALIFORNIA', 'USA']
445
['PENNSYLVANIA', '

['MASSACHUSETTS', 'USA']
805
['SACRAMENTO', 'CA']
806
['HOUSTON', 'TX']
807
['MATA']
808
['CYPRESS', 'CA']
809
['TACOMA', 'WA']
810
['BRITISH']
811
[]
812
['LONG', 'BEACH', 'CA']
813
['GUMPOLDSKIRCHEN', 'ÖSTERREICH']
814
['DENVER', 'CO']
815
['LOS ANGELES', 'CA']
816
['MESA', 'ARIZONA']
817
['ITHACA', 'NY']
818
['UPPER', 'EAST', 'MANHATTAN']
819
['TEXAS']
820
['THE', 'BAG']
821
['OKLAHOMA', 'CITY', 'OKLA']
822
['PA']
823
['BRITISH']
824
['NORMAN', 'OK']
825
['NOLA']
826
['WACO', 'TX']
827
['BATON']
828
[]
829
['SAN', 'BRUNO', 'CA']
830
['NEW YORK']
831
['CHICAGO', 'IL']
832
['UNIVERSE']
833
['CHICAGO', 'HOUSTON']
834
['JACKSONVILLE', 'BEACH', 'FL']
835
[]
836
['CHICAGO', 'IL']
837
[]
838
['NAP', 'TOWN', 'USA']
839
['MULTNOMAH', 'COUNTY', 'OREGON']
840
['ARIZONA', 'USA']
841
['UNITED STATES']
842
['FARGO', 'ND']
843
['JERSEY', 'USA']
844
['THE', 'THE']
845
['FLORIDA', 'USA']
846
['TEMPE', 'LIVING']
847
['CHICAGO', 'IL']
848
['COLORADO', 'USA']
849
['PARK', 'CITY', 'UTAH']
850
['MILL', '

['SUNSHINE', 'COAST']
1186
['OR']
1187
['AUSTIN', 'TX']
1188
['SAN', 'ANTONIO', 'TX']
1189
['PORT', 'RICHEY', 'FL']
1190
['UTAH', 'USA']
1191
['NAPA', 'CA']
1192
['SAN', 'ANTONIO', 'TX']
1193
['NEW YORK', 'NY']
1194
['JOKELAND', 'CA']
1195
['NEW YORK', 'USA']
1196
['SAN', 'DIEGO', 'CA']
1197
['LOS ANGELES']
1198
['TENNESSEE', 'USA']
1199
['NEW YORK', 'NY']
1200
['CA']
1201
['WILMINGTON']
1202
['SOUTHAVEN']
1203
['NEW YORK']
1204
['TALLAHASSEE', 'FL']
1205
['STATEN', 'ISLAND', 'NYC']
1206
['BOSTON', 'MA']
1207
['SEATTLE', 'WA']
1208
['CALAMBA CITY', 'CALABARZON']
1209
['NEW YORK', 'NY']
1210
['ENGLEWORLD', 'USA']
1211
['PETAL']
1212
['SOUTH', 'FLORIDA']
1213
['GR']
1214
['COLORADO']
1215
['MARYLAND', 'USA']
1216
['HOUSTON', 'TX']
1217
['ANNOYED']
1218
['SEATTLE']
1219
['KOLLEGE']
1220
['TWITCH', 'TIKTOK', 'YOUTUBE']
1221
['PALM', 'BEACH', 'FL']
1222
['HAZEL', 'GREEN', 'AL']
1223
['ANY']
1224
['MARIETTA', 'GA']
1225
['VIRGINIA', 'BEACH', 'VA']
1226
['ATLANTA', 'GA']
1227
['CHICAGO', 'IL'

['HOUSTON', 'TX']
1561
['AUSTIN', 'TX']
1562
[]
1563
['SAN', 'DIEGO', 'CA']
1564
['SEATTLE', 'WA']
1565
['CAROLINA', 'USA']
1566
['EUGENE', 'OR']
1567
['CASTLE', 'VON']
1568
['TORONTO', 'CANADA']
1569
['DUCK', 'KEY', 'FL']
1570
['SEATTLE', 'WA']
1571
['BOSTON', 'AND']
1572
['SACRAMENTO', 'CALIFORNIA']
1573
['SACRAMENTO', 'CA']
1574
['ORLANDO', 'FL']
1575
['WA']
1576
['BOCA', 'RATON', 'FL']
1577
['AUSTIN', 'TX']
1578
['GOTHAM', 'CITY']
1579
['HARRISBURG', 'PA']
1580
['SAN', 'DIEGO', 'CA']
1581
['BURBANK', 'CA']
1582
['NORTH', 'LAS', 'NV']
1583
['KY']
1584
['DAD', 'HOUSE']
1585
['LOS ANGELES', 'CA']
1586
['BC']
1587
['MILKY', 'WAY', 'VIRGO']
1588
['CHICAGO', 'IL']
1589
['LOS ANGELES']
1590
['ATLANTA', 'GA']
1591
['SPRING', 'HILL', 'FL']
1592
['NASHVILLE', 'TN']
1593
['LADERA', 'HEIGHTS', 'CA']
1594
['CHICAGO']
1595
['UNITED STATES']
1596
['PORTUGAL']
1597
['NV']
1598
['ORLANDO', 'FL']
1599
['HOUSTON', 'TX']
1600
['SAN', 'FRANCISCO']
1601
['ONLINE']
1602
['NEW YORK', 'NY']
1603
['SAN', 'A

['SCOTTSDALE', 'AZ']
1936
['LOS ANGELES']
1937
[]
1938
[]
1939
['RIVERSIDE', 'CA']
1940
['LOS ANGELES', 'CA']
1941
['RANDALLSTOWN', 'MD']
1942
['DALLAS']
1943
['MIAMI', 'FL']
1944
[]
1945
['AR']
1946
['ATL']
1947
['PITTSBURGH', 'PA']
1948
['HOLLYWOOD', 'LOS']
1949
['ANAHEIM', 'CA']
1950
['PORTLAND', 'OREGON']
1951
['YEE', 'HAW', 'ND']
1952
['LOS ANGELES', 'CA']
1953
['ALEXANDRIA', 'VA']
1954
['HOT', 'SPRINGS', 'AR']
1955
[]
1956
['NEW YORK', 'USA']
1957
['HOUSTON', 'TX']
1958
['NORTH', 'TEXAS']
1959
['SAN', 'FRANCISCO', 'CA']
1960
['CONYERS', 'ATLANTA']
1961
['SAN', 'MARCOS', 'TX']
1962
['LEGO']
1963
['LEHIGH', 'ACRES', 'FL']
1964
['DALLAS', 'TX']
1965
['GRAND PRAIRIE', 'TX']
1966
['RIPLEY']
1967
['LUBBOCK', 'TX']
1968
['FORT', 'WORTH', 'TX']
1969
['INDIANA', 'USA']
1970
['BUSHWICK', 'BROOKLYN', 'NY']
1971
['STATE', 'COLLEGE', 'PA']
1972
['IDAHO', 'USA']
1973
['HOUSTON']
1974
['MINNESOTA']
1975
['PITTSBURGH', 'PA']
1976
['MARIETTA', 'GA']
1977
['SAN', 'BERNARDINO', 'CALIFORNIA']
1978
[

['BOOBS', 'ADMIRATION']
2316
['TEXAS', 'USA']
2317
['CHICAGO', 'IL']
2318
['LOS ANGELES', 'CA']
2319
['SEATTLE', 'WA']
2320
['BAHAMAS']
2321
['FLORIDA', 'USA']
2322
['BOA']
2323
[]
2324
[]
2325
['MOSHPIT']
2326
['OHIO', 'USA']
2327
['CLINTON', 'NJ']
2328
['SEATTLE', 'WA']
2329
['HOU', 'TX']
2330
['BROOKLYN', 'NY']
2331
['CRAIG', 'COLORADO']
2332
['HOUSTON', 'TX']
2333
['CINCINNATI', 'OH']
2334
['LOS ANGELES', 'CA']
2335
['PRINCETON', 'KY']
2336
['TUCSON', 'AZ']
2337
['KANSAS', 'CITY']
2338


0                                    []
1                       [CLEVELAND, OH]
2                          [HARRISBURG]
3       [HOUSTON, TX, USA, THE, GHETTO]
4                          [DENVER, CO]
                     ...               
2333                   [CINCINNATI, OH]
2334                  [LOS ANGELES, CA]
2335                    [PRINCETON, KY]
2336                       [TUCSON, AZ]
2337                     [KANSAS, CITY]
Name: user_location2, Length: 2338, dtype: object

In [21]:
len(df['user_location2'][0])
df2 = df[df['user_location2'].apply(lambda x: len(x) > 0)].reset_index()
df2

,level_0,index,user_id,user_location,user_bio,hashtag,tokens,Segmented#,user_location2
0,1,1,1.180000e+18,"CLEVELAND, OH","I'm a nerdy guy who loves art, music, wrestlin...",[],nerdy guy love art music wrestle comics poly,NaN,"[CLEVELAND, OH]"
1,2,2,1.440000e+18,HARRISBURG PA,No worries Lil Ms Sunshine cause I don't mess ...,[],worry lil sunshine cause mess woman moon civil...,NaN,[HARRISBURG]
2,3,3,3.169218e+09,"HOUSTON, TX, USA, THE GHETTO","Fuck yo feelings!\nFollow me please, I follow ...","['lgbtq', 'feminist', 'blacklivesmatter', 'Army']",fuck feel follow follow atl bear nigerian rais...,lgbtq feminist blacklivesmatter army,"[HOUSTON, TX, USA, THE, GHETTO]"
3,4,4,1.219720e+08,"DENVER, CO",I am a huge sports and music fan. I graduated ...,[],huge sport music fan graduate regis university...,NaN,"[DENVER, CO]"
4,6,6,2.349053e+07,UTAH,I'm a writer and a cult film fanatic who loves...,[],writer cult film fanatic love politics,NaN,[UTAH]
...,...,...,...,...,...,...,...,...,...
2084,2333,4105,2.961228e+08,"CINCINNATI, OH",Christian. Husband. Father. Ph.D. Candidate @B...,['ChelseaFC'],christian husband father candidate mosquito re...,chelsea fc,"[CINCINNATI, OH]"
2085,2334,4106,8.800000e+17,"LOS ANGELES, CA","MarTech, MBA, Foodie, Bilingual, Soccer, Boxin...",['FairPlay'],martech mba foodie bilingual soccer box gamer ...,fair play,"[LOS ANGELES, CA]"
2086,2335,4111,1.153059e+09,"PRINCETON, KY","streams occasionally, will choose race cars ov...",[],stream occasionally choose race cars important...,NaN,"[PRINCETON, KY]"
2087,2336,4116,2.432869e+08,"TUCSON, AZ",Enjoy the practical wisdom of daily life. Prom...,[],enjoy practical wisdom daily life promote trut...,NaN,"[TUCSON, AZ]"


In [28]:

def replace_states(column):
    statez = us.states.mapping('abbr', 'name')
    lst_of_abbrv = list(statez.keys())
    lst_of_states = list(statez.values())
    lst_of_states = [each_string.upper() for each_string in lst_of_states]

    
    state_cell = []
    for cell in column:
        state_1 = []
        for item in cell:
            state = ""
            if item not in lst_of_states:
                state =  item
            else:
                position = lst_of_states.index(item)
                state = lst_of_abbrv[position]
            state_1.append(state)
        state_cell.append(state_1)
    return state_cell

def replace_abbrv(column):
    statez = us.states.mapping('abbr', 'name')
    lst_of_abbrv = list(statez.keys())
    lst_of_states = list(statez.values())
    lst_of_states = [each_string.upper() for each_string in lst_of_states]

    
    state_cell = []
    for cell in column:
        state_1 = []
        for item in cell:
            state = ""
            if item not in lst_of_abbrv:
                state =  ""
            else:
                position = lst_of_abbrv.index(item)
                state = lst_of_abbrv[position]
            state_1.append(state)
        state_cell.append(state_1)
    return state_cell


df2['user_location2'] = replace_states(df2['user_location2'])
df2['user_location2'] = replace_abbrv(df2['user_location2'])
df2['user_location2']

['', '']

In [29]:
def clean_states (column):
    states = []
    for cell in column:
        strings = cell
        listToStr = ''.join(map(str, strings))
        states.append(listToStr)
    return states

df2['user_location2'] = clean_states(df2['user_location2'])
df2['user_location2']

0        
1        
2        
3        
4        
       ..
2084     
2085     
2086     
2087     
2088     
Name: user_location2, Length: 2089, dtype: object

In [24]:
df2

,level_0,index,user_id,user_location,user_bio,hashtag,tokens,Segmented#,user_location2,user_location6
0,1,1,1.180000e+18,"CLEVELAND, OH","I'm a nerdy guy who loves art, music, wrestlin...",[],nerdy guy love art music wrestle comics poly,NaN,"[, OH]",OH
1,2,2,1.440000e+18,HARRISBURG PA,No worries Lil Ms Sunshine cause I don't mess ...,[],worry lil sunshine cause mess woman moon civil...,NaN,[],
2,3,3,3.169218e+09,"HOUSTON, TX, USA, THE GHETTO","Fuck yo feelings!\nFollow me please, I follow ...","['lgbtq', 'feminist', 'blacklivesmatter', 'Army']",fuck feel follow follow atl bear nigerian rais...,lgbtq feminist blacklivesmatter army,"[, TX, , , ]",TX
3,4,4,1.219720e+08,"DENVER, CO",I am a huge sports and music fan. I graduated ...,[],huge sport music fan graduate regis university...,NaN,"[, CO]",CO
4,6,6,2.349053e+07,UTAH,I'm a writer and a cult film fanatic who loves...,[],writer cult film fanatic love politics,NaN,[UT],UT
...,...,...,...,...,...,...,...,...,...,...
2084,2333,4105,2.961228e+08,"CINCINNATI, OH",Christian. Husband. Father. Ph.D. Candidate @B...,['ChelseaFC'],christian husband father candidate mosquito re...,chelsea fc,"[, OH]",OH
2085,2334,4106,8.800000e+17,"LOS ANGELES, CA","MarTech, MBA, Foodie, Bilingual, Soccer, Boxin...",['FairPlay'],martech mba foodie bilingual soccer box gamer ...,fair play,"[, CA]",CA
2086,2335,4111,1.153059e+09,"PRINCETON, KY","streams occasionally, will choose race cars ov...",[],stream occasionally choose race cars important...,NaN,"[, KY]",KY
2087,2336,4116,2.432869e+08,"TUCSON, AZ",Enjoy the practical wisdom of daily life. Prom...,[],enjoy practical wisdom daily life promote trut...,NaN,"[, AZ]",AZ


In [31]:
df2.to_csv(r'data/step3_preprocessed_bios_plus_states.csv', index = False,   
                 header=True)